# Descrição Dataframe

## Amostra
Segue uma pequena amostra do dataframe já tratado

In [17]:
import pandas as pd

df = pd.read_csv("dados/amostra.csv")

df["dt_despacho_pedido"] = pd.to_datetime(
    df["dt_despacho_pedido"] + " " + df["hr_despacho_pedido"],
    format="%Y-%m-%d %H:%M:%S",
)

df["dt_entrega_pedido"] = pd.to_datetime(
    df["dt_entrega_pedido"] + " " + df["hr_entrega_pedido"],
    format="%Y-%m-%d %H:%M:%S",
)

df["dt_previsao_entrega_cliente"] = pd.to_datetime(df["dt_previsao_entrega_cliente"])

df["dt_criacao"] = pd.to_datetime(df["dt_criacao"])

df["dt_pagamento_pedido"] = pd.to_datetime(df["dt_pagamento_pedido"])

df["flg_existem_ocorrencias"] = df["flg_existem_ocorrencias"].str.lower() == "sim"

df["entregue_no_prazo"] = df["tp_performance_entrega"] == "Entregue no Prazo"

df["qtde_dias_prazo_entrega"] = df["dt_previsao_entrega_cliente"] - df["dt_despacho_pedido"]

df["qtde_dias_efet_entrega"] = df["dt_entrega_pedido"] - df["dt_despacho_pedido"]

df = df.query("not dt_despacho_pedido.isnull() and not dt_entrega_pedido.isnull() and not dt_pagamento_pedido.isnull()")

df.drop(
    columns=[
        "Unnamed: 0",
        "hr_despacho_pedido",
        "hr_entrega_pedido",
        "tp_performance_entrega",
        "cod_pedido"
    ],
    inplace=True,
)

df

,cidade_destinatario,uf,grp_transportadora,dt_despacho_pedido,dt_entrega_pedido,dt_previsao_entrega_cliente,dt_criacao,dt_pagamento_pedido,flg_existem_ocorrencias,tp_praca,des_unidade_negocio,des_cd_origem,qtd_dias_tat,entregue_no_prazo,qtde_dias_prazo_entrega,qtde_dias_efet_entrega
0,CURITIBA,PR,Transportadora 4,2023-12-04 05:56:39,2023-12-07 12:19:04,2023-12-07,2023-11-30,2023-11-30,False,Capital,Multi,PR-Campina G. Sul,5.0,True,2 days 18:03:21,3 days 06:22:25
1,PARAUAPEBAS,PA,Transportadora 2,2023-11-21 12:39:55,2023-12-01 12:23:47,2023-12-11,2023-11-15,2023-11-17,False,Interior,Mono,PR-Campina G. Sul,10.0,True,19 days 11:20:05,9 days 23:43:52
2,JACAREZINHO,PR,Transportadora 4,2023-10-28 10:12:24,2023-10-31 16:10:01,2023-11-06,2023-10-26,2023-10-27,False,Interior,Mono,PR-Campina G. Sul,2.0,True,8 days 13:47:36,3 days 05:57:37
3,SÃO PAULO,SP,Transportadora 5,2023-08-23 17:52:06,2023-08-24 12:00:13,2023-08-24,2023-08-22,2023-08-22,False,Capital,Multi,PR-Campina G. Sul,2.0,True,0 days 06:07:54,0 days 18:08:07
4,SAO LUIS,MA,Transportadora 3,2023-08-14 08:41:00,2023-08-23 15:22:59,2023-08-24,2023-08-13,2023-08-12,False,Capital,Multi,PR-Campina G. Sul,8.0,True,9 days 15:19:00,9 days 06:41:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503523,BELO HORIZONTE,MG,Transportadora 3,2023-09-04 15:43:43,2023-09-07 10:33:03,2023-09-06,2023-09-03,2023-09-03,True,Capital,Multi,PR-Campina G. Sul,3.0,False,1 days 08:16:17,2 days 18:49:20
503525,SÃO PAULO,SP,Transportadora 5,2023-08-23 17:39:03,2023-08-26 12:30:53,2023-08-24,2023-08-22,2023-08-22,True,Capital,Multi,PR-Campina G. Sul,3.0,False,0 days 06:20:57,2 days 18:51:50
503526,São Paulo,SP,Transportadora 1,2023-09-19 15:09:15,2023-09-25 14:03:59,2023-09-20,2023-09-18,2023-09-16,True,Capital,Multi,SP-Jaguaré,6.0,False,0 days 08:50:45,5 days 22:54:44
503527,NITERÓI,RJ,Transportadora 3,2023-10-16 14:59:28,2023-10-19 14:56:27,2023-10-18,2023-10-14,2023-10-14,False,Reg. Metropolitana,Mono,SP-Registro,4.0,False,1 days 09:00:32,2 days 23:56:59


## Tratamentos
Segue tratamentos efetuados em cima do dataset:

- Foi retirado do dataframe os registros cuja **data despacho** esteja vazia.
- Foi retirado do dataframe os registros cuja **data entrega** esteja vazia.
- Foi retirado do dataframe os registros cuja **data pagamento** esteja vazia.
- Os campos contendo **horas** foram aglutinados nos seus respectivos campos de data.
  
## Descrição das colunas

- `cidade_destinatario`: Cidade de destino do pedido.
- `uf`: Estado da cidade destino.
- `grp_transportadora`: (Grupo) transportado que está realizando a entrega.
- `dt_despacho_pedido`: data e hora em que o pedido foi despachado.
- `dt_entrega_pedido`: data e hora em que o pedido foi entregue.
- `dt_previsao_entrega_cliente`: data prevista para a entrega do pedido.
- `dt_criacao`: data de cria o do pedido.
- `dt_pagamento_pedido`: data do pagamento do pedido.
- `flg_existem_ocorrencias`: booleano que indica se houve ocorrencias durante a entrega do pedido (problema no andamento da entrega).
- `tp_praca`: Tipo de praça de entrega (Capital, Interior, Região Metropolitana).
- `des_unidade_negocio`: Classificação da unidade de negócio.
- `des_cd_origem`: Centro de distribuição de origem.
- `entregue_no_prazo`: booleano que indica se o pedido foi entregue no prazo.
- `qtde_dias_prazo_entrega`: quantidade de dias entre a **data de despacho** e a **data de entrega prevista**.
- `qtde_dias_efet_entrega`: quantidade de dias entre a **data de despacho** e a **data de entrega efetiva**.

# Análise Exploratória

## Entendimento entregas atrasadas sobre espaço temporal 

In [ ]:
import hvplot.pandas

df_atrasos_meses = (
    df.query("entregue_no_prazo == False")[["dt_entrega_pedido"]]
    .set_index("dt_entrega_pedido")
    .groupby(pd.Grouper(freq="ME"))
    .size()
)

df_atrasos_meses.hvplot.bar(
    title="Ocorrências Mensais de Atrasos",
    xlabel="Data",
    ylabel="Quantidade de Eventos",
)

:Bars   [dt_entrega_pedido]   (0)